# Projeto 1 - Ciência dos Dados

Nome: Carlos Andrade Inacio

Nome: Carlos Felipe Borges Mesquita

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\CarlosInacio\Documents\code\insper\cdados\Projeto 1\cdados-netflix


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'netflix.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@zanchettagui :(\nah no show dele do netflix e...,0
1,"jojo vai chegar na netflix mês q vem, será q e...",1
2,"com geladeira cheia, conta no banco positiva, ...",0
3,estou rachando o bico com o filme cabras da pe...,1
4,@twolana9 netflix \nforças complex,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,1. não\n2. 1\n3. peixes\n4. não\n5. apegada\n6...,0
1,@jpncunha1397 @arrascabigol1 @diegooliveiira_ ...,0
2,@netflixbrasil apanha muito também ela né netf...,0
3,eu estava sem nenhuma série pra assistir e ach...,0
4,@fraannnnnnnnn final de temporada nas séries d...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

 - Relevante: opinião referente ao conteúdo na plataforma.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
################################################################################################################################
################################################################################################################################
########################         NÃO ESQUECER DE ALTERAR A QUANTIDADE DE TWEETS PROCESSADOS       ##############################
################################################################################################################################
################################################################################################################################


def find_instance(start_str, end_str, string):
    start = string.find(start_str)
    end = string.find(end_str, start)
    instance = ''
    if end < start or end == None:
        instance = string[start:]
    else:
        instance = string[start:end + 1]
        
    return instance

def cleanup(serie, filtro = ''):
    lista_tweets = []
    punctuation = '[!-.:?;,]'
    pattern = re.compile(punctuation)

    tweets = []
    if filtro:
        tweets = serie[filtro]
    else:
        tweets = serie
    
    for texto in tweets:
        # removendo emojis e pontuação
        tweet = emoji.get_emoji_regexp().sub(u'', texto)        
        tweet = re.sub(pattern, '', tweet)
        
        while '\n' in tweet or '  ' in tweet or '"' in tweet or "'" in tweet:
            # removendo newline
            tweet = tweet.replace('\n', '')
            
            # mesmo que existam mais de 2 espaços de uma vez só, o while loop vai cortando o número de espaços
            # até se tornar um único espaço
            tweet = tweet.replace('  ', ' ')
            
            # removendo aspas
            tweet = tweet.replace('"', '')
            tweet = tweet.replace("'", '')
        
        while 'http' in tweet:
            # removendo urls
            url = find_instance('http', ' ', tweet)          
            if url:
                tweet = tweet.replace(url, '')
        
        while '@' in tweet:
            # removendo menções
            mention = find_instance('@', ' ', tweet)
            if mention:
                tweet = tweet.replace(mention, '')
            
        lista_tweets.append(tweet)
        
    return lista_tweets


relevantes = train.loc[(train['Classificação'] == 1), :]["Treinamento"]
irrelevantes = train.loc[(train['Classificação'] == 0), :]["Treinamento"]
totais = train.loc[:, ["Treinamento", "Classificação"]]

tweets_relevantes = cleanup(relevantes)
tweets_irrelevantes = cleanup(irrelevantes)
tweets_totais = cleanup(totais, "Treinamento")

palavras_relevantes = []
palavras_irrelevantes = []
palavras_totais = []

for tweet in tweets_relevantes:
    palavras_relevantes += tweet.split()
    palavras_totais += tweet.split()
for tweet in tweets_irrelevantes:
    palavras_irrelevantes += tweet.split()
    palavras_totais += tweet.split()
    
serie_relevantes = pd.Series(palavras_relevantes)
serie_irrelevantes = pd.Series(palavras_irrelevantes)
serie_totais = pd.Series(palavras_irrelevantes)
    
palavras_relevantes_relativas = serie_relevantes.value_counts(True)
palavras_irrelevantes_relativas = serie_irrelevantes.value_counts(True)
palavras_totais_relativas = serie_totais.value_counts(True)

In [7]:
def probabilidade_tweet(_palavras, tabela_relativa, relevantes = False):
    probabilidade = 0
    for i in range(len(_palavras)):
        palavra = _palavras[i]
        if not palavra in tabela_relativa:
            continue
            
        probabilidade_palavra = tabela_relativa[palavra]
        
        if i == 0:
            probabilidade = probabilidade_palavra
        else:
            probabilidade *= probabilidade_palavra
    
    return probabilidade

def definir_relevancia(tweet):
    prob_relevante = len(serie_relevantes) / len(serie_totais)
    prob_irrelevante = len(serie_irrelevantes) / len(serie_totais)

    palavras = tweet.split()
    
    prob_tweet_dado_relevante = probabilidade_tweet(palavras, palavras_relevantes_relativas, True)
    prob_tweet_dado_irrelevante = probabilidade_tweet(palavras, palavras_irrelevantes_relativas)

    prob_relevante_tweet = prob_tweet_dado_relevante * prob_relevante
    prob_irrelevante_tweet = prob_tweet_dado_irrelevante * prob_irrelevante

    if prob_relevante_tweet > prob_irrelevante_tweet:
        return 1
    elif prob_relevante_tweet < prob_irrelevante_tweet:
        return 0
    else:
        return '??'


In [9]:
resultados = {"Tweet" : [], "Classificação" : []}

for tweet in tweets_totais:
    resultados["Tweet"].append(tweet)
    resultados["Classificação"].append(definir_relevancia(tweet))
    
resultados_df = pd.DataFrame.from_dict(resultados)

acertos = 0
erros = 0
nao_identificados = 0

for indice in range(len(resultados_df)):
    if resultados_df.iloc[[indice]]["Classificação"].item() == totais.iloc[[indice]]["Classificação"].item():
        acertos += 1
        continue
    
    if resultados_df.iloc[[indice]]["Classificação"].item() == "??":
        nao_identificados += 1
        continue
    
    erros += 1
    
acerto_porcentagem = (100 / (acertos + erros)) * acertos
erro_porcentagem = (100 / (acertos + erros)) * erros

print(f'% acerto: {acerto_porcentagem}\n% erro: {erro_porcentagem}\nnão identificados: {nao_identificados}')

% acerto: 32.77591973244147
% erro: 67.22408026755853
não identificados: 0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
totais_teste = test.loc[:,["Teste", "Classificação"]]

tweets_teste = cleanup(totais_teste, "Teste")

In [12]:
resultados = {"Tweet" : [], "Classificação" : []}

for tweet in tweets_teste:
    resultados["Tweet"].append(tweet)
    resultados["Classificação"].append(definir_relevancia(tweet))
    
resultados_df = pd.DataFrame.from_dict(resultados)

acertos = 0
erros = 0
nao_identificados = 0

for indice in range(len(resultados_df)):
    if resultados_df.iloc[[indice]]["Classificação"].item() == totais_teste.iloc[[indice]]["Classificação"].item():
        acertos += 1
        continue
    
    if resultados_df.iloc[[indice]]["Classificação"].item() == "??":
        nao_identificados += 1
        continue
    
    erros += 1
    
acerto_porcentagem = (100 / (acertos + erros)) * acertos
erro_porcentagem = (100 / (acertos + erros)) * erros

print(f'% acerto: {acerto_porcentagem}\n% erro: {erro_porcentagem}\nnão identificados: {nao_identificados}')
    

% acerto: 48.55491329479769
% erro: 51.445086705202314
não identificados: 28


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**